In [3]:
#Importar librerías para tratamiento de datos
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
import re
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

from selenium.common.exceptions import NoSuchElementException

In [6]:
#Creamos la función

def get_data(driver,xpath):
    try:
        data = driver.find_element("xpath", xpath).text
    except:
        data = "N/A"
    return data


def extraer_datos_actores(nombre_archivo):
    df_pelis = pd.read_csv("peliculas_api.csv")

    df_pelis["ID de la Película"] = df_pelis["ID de la Película"].astype(str)

    driver = webdriver.Chrome()
    driver.maximize_window()

    for id_imdb in df_pelis["ID de la Película"][:10]:
        try:
            url = f"https://www.imdb.com/title/{id_imdb}/"
            driver.get(url)

            sleep(2)
            try:
                driver.find_element("xpath", '//*[@id="__next"]/div/div/div[2]/div/button[2]').click()
            except NoSuchElementException:
                pass 

            sleep(2) #verificar si este sleep es util o no
            driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[1]/div/div[2]/ul/li[1]/a').click()
    
            sleep(2)

            lista_actores = []

            nombre_peli = get_data(driver,'//*[@id="main"]/div[1]/div[1]/div/div[1]/h3/a')
            
            for i in range(2,12):
                try:

                    sleep(2)

                    driver.find_element("xpath",f'//*[@id="fullcredits_content"]/table[3]/tbody/tr[{i}]/td[2]/a').click()  
                    sleep(3)

                    nombre_actxr = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/h1/span').text

                    conocido_por1 = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[1]/div[2]/div[1]/a').text
                    try:
                        conocido_por2 = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/div[2]/section[1]/div[2]/div/div[2]/div[2]/div[2]/div[1]/a').text
                    except:
                        conocido_por2 = "N/A"    
                    conocido_por = [conocido_por1, conocido_por2] if conocido_por2 != "N/A" else [conocido_por1]
                
                    try:
                        fecha_nacimiento = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div/div[2]/section/aside/div/span[2]').text
                    except:
                        fecha_nacimiento = "N/A"
                    year_nacimiento = fecha_nacimiento.split(" ")[-1:][0] if fecha_nacimiento != "N/A" else "N/A"

                    try:
                        profesiones = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text.split("\n")
                    except:
                        profesiones = "N/A"
                    profesiones_string = ", ".join(profesiones) if profesiones != "N/A" else "N/A"

                    try:
                        premios = driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul/li/span').text
                    except:
                        premios = "N/A"

                    info_actores = (nombre_peli,nombre_actxr,conocido_por,year_nacimiento,profesiones_string,premios)
                    lista_actores.append(info_actores)

                except NoSuchElementException:
                    print("pasa algo aqui")
                    pass

                finally:
                    driver.back()
            
            print("Datos de los actores:")
            print(lista_actores)

        except Exception as e:
            print(f"Error al procesor la película con ID de IMDb {id_imdb}: {e}")

    driver.quit()

In [7]:
extraer_datos_actores("peliculas_api.csv")

Datos de los actores:
[('Pál Adrienn', 'Éva Gábor', ['Europa, East', 'Pál Adrienn'], 'N/A', 'Actress', 'N/A'), ('Pál Adrienn', 'István Znamenák', ['The Citizen', 'Taxidermia'], '1966', 'Actor, Director, Cinematographer', '1 win'), ('Pál Adrienn', 'Ákos Horváth', ['Control', 'Haunting Trophies'], '1968', 'Actor, Writer', '3 wins & 1 nomination'), ('Pál Adrienn', 'Lia Pokorny', ['A La Carte', 'Well'], '1971', 'Actress', 'N/A'), ('Pál Adrienn', 'Izabella Hegyi', ['Friss levegö', 'Pál Adrienn'], 'N/A', 'Actress', 'N/A'), ('Pál Adrienn', 'Szabolcs Thuróczy', ['Kills On Wheels', 'Sandwich'], '1971', 'Actor, Writer', 'N/A'), ('Pál Adrienn', 'Ági Margitai', ['Jakob the Liar', 'Ripacsok'], '1937', 'Actress', 'Agi Margitai'), ('Pál Adrienn', 'Eszter Márton', ['Sose halunk meg', 'Jóban rosszban'], '1969', 'Writer, Actress', 'February 28, 1969'), ('Pál Adrienn', 'Lóránt Matta', ['Pál Adrienn', 'Mi kérünk elnézést!'], '1973', 'Actor', 'August 12, 1973'), ('Pál Adrienn', 'Róbert Kardos', ['Last Chan